In [1]:
import getpass
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_groq import ChatGroq
from dotenv import load_dotenv

In [2]:
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = os.getenv('LANGCHAIN_TRACING_V2')
os.environ['LANGCHAIN_ENDPOINT'] = os.getenv('LANGCHAIN_ENDPOINT')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')

In [3]:
from langchain_core.tools import tool

In [4]:
@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiplica dos números enteros."""
    return first_int * second_int

In [5]:
llm = ChatGroq(temperature=0, model="llama3-70b-8192")

In [6]:
llm_with_tools = llm.bind_tools([multiply])

In [7]:
msg = llm_with_tools.invoke("¿Cuanto es 5 veces cuarenta y dos?")
msg

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_15v2', 'function': {'arguments': '{"first_int":5,"second_int":42}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_time': 0.222, 'completion_tokens': 68, 'prompt_time': 0.678, 'prompt_tokens': 927, 'queue_time': None, 'total_time': 0.9, 'total_tokens': 995}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_9625f32860', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-97f29326-8a4b-44ed-9cc4-3a53e70cde30-0', tool_calls=[{'name': 'multiply', 'args': {'first_int': 5, 'second_int': 42}, 'id': 'call_15v2'}])

In [8]:
call = msg.tool_calls
call

[{'name': 'multiply',
  'args': {'first_int': 5, 'second_int': 42},
  'id': 'call_15v2'}]

In [9]:
from operator import itemgetter

In [10]:
chain = llm_with_tools | (lambda x: x.tool_calls[0]["args"]) | multiply
chain

RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001376214A800>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000137621EAFE0>, model_name='llama3-70b-8192', temperature=1e-08, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'multiply', 'description': 'multiply(first_int: int, second_int: int) -> int - Multiplica dos números enteros.', 'parameters': {'type': 'object', 'properties': {'first_int': {'type': 'integer'}, 'second_int': {'type': 'integer'}}, 'required': ['first_int', 'second_int']}}}]})
| RunnableLambda(lambda x: x.tool_calls[0]['args'])
| StructuredTool(name='multiply', description='multiply(first_int: int, second_int: int) -> int - Multiplica dos números enteros.', args_schema=<class 'pydantic.v1.main.multiplySchema'>, func=<function multiply at 0x0000013761C9CDC0>)

In [11]:
resultado = chain.invoke("¿Cuanto es 4 veces 23?")
resultado

92